# 分散表現の確認
word2vecによる分散表現について学びます。

## コーパスの前処理
前のセクションと同様に、コーパスに前処理を行います。

In [1]:
import re
import pickle
from janome.tokenizer import Tokenizer

with open("wagahaiwa_nekodearu.txt", mode="r", encoding="utf-8") as f:  # ファイルの読み込み
    wagahai_original = f.read()

wagahai = re.sub("《[^》]+》", "", wagahai_original) # ルビの削除
wagahai = re.sub("［[^］]+］", "", wagahai) # 読みの注意の削除
wagahai = re.sub("[｜ 　「」\n]", "", wagahai) # | と全角半角スペース、「」と改行の削除

seperator = "。"  # 。をセパレータに指定
wagahai_list = wagahai.split(seperator)  # セパレーターを使って文章をリストに分割する
wagahai_list.pop() # 最後の要素は空の文字列になるので、削除
wagahai_list = [x+seperator for x in wagahai_list]  # 文章の最後に。を追加
        
t = Tokenizer()

wagahai_words = []
for sentence in wagahai_list:
    wagahai_words.extend(list(t.tokenize(sentence, wakati=True)))
   # 文章ごとに単語に分割し、リストに格納
    
with open('wagahai_words.pickle', mode='wb') as f:  # pickleに保存
    pickle.dump(wagahai_words, f)

保存できていることを確認します。

In [2]:
with open('wagahai_words.pickle', mode='rb') as f:
    wagahai_words = pickle.load(f)

# print(wagahai_words)

## word2vecを用いた学習
今回はword2vecのためにライブラリgensimを使います。  
gensimは、様々なトピックモデルを実装したPythonライブラリです。  
トピックモデルとは、潜在的なトピックから文章が確率的に生成されると仮定したモデルです。

gensimについて、詳細は以下のリンクを参考にどうぞ。  
https://radimrehurek.com/gensim/

以下では、word2vecを用いてコーパスの学習を行い、学習済みのモデルを作成します。

In [3]:
from gensim.models import word2vec

# size : 中間層のニューロン数
# min_count : この値以下の出現回数の単語を無視
# window : 対象単語を中心とした前後の単語数
# iter : epochs数
# sg : skip-gramを使うかどうか 0:CBOW 1:skip-gram
model = word2vec.Word2Vec(wagahai_words, vector_size=100, min_count=5, window=5, epochs=20, sg = 0)

/opt/anaconda3/envs/local/lib/python3.9/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


分散表現を見ていきましょう。  
重みを表す行列（分散表現）の形状と、行列そのものを表示します。

In [4]:
print(model.wv.vectors.shape)  # 分散表現の形状
print(model.wv.vectors)  # 分散表現

(1918, 100)
[[ 0.1942895  -0.0501147  -0.12820184 ...  0.48983166 -0.19787253
   0.44540128]
 [-0.7643005  -0.6531269   0.73792166 ...  0.93104035 -0.53917426
  -0.40874007]
 [ 0.09456427  0.5060688  -0.35848436 ...  0.9517323   0.3165773
  -0.10096911]
 ...
 [-0.0058881  -0.03270137 -0.00377174 ... -0.08479176 -0.06973612
  -0.12364889]
 [-0.03240474 -0.01616859 -0.0747441  ... -0.12825122  0.19710721
  -0.19638833]
 [-0.00867903  0.08252893 -0.01295252 ... -0.14270966  0.10225292
  -0.13844164]]


語彙の数、および語彙の最初の10語を表示します。

In [5]:
print(len(model.wv.index_to_key))  # 語彙の数
print(model.wv.index_to_key[:10])  # 最初の10単語

1918
['の', 'い', 'る', 'て', 'な', 'と', 'に', '。', 'か', 'し']


語彙における、最初の単語の単語ベクトルを2通りの方法で表示します。

In [6]:
print(model.wv.vectors[0])  # 最初のベクトル
print(model.wv.__getitem__("の"))  # 最初の単語「の」のベクトル

[ 0.1942895  -0.0501147  -0.12820184  0.13635032 -0.6053373   0.5225144
 -0.01715966 -0.01865283 -0.9644791   0.18599837 -0.01022938 -0.33068272
  0.8365034   0.57771134 -0.61078745 -0.45551297  0.9060428   0.6509636
 -0.7758635  -0.98626477  0.5622609  -0.60753447  0.27126592  0.21536641
  0.2914974   0.10955942 -0.6375244   0.57521844 -0.4587857   0.14071426
  0.18125032  0.18191835  0.00340088 -0.8161173  -0.39108387  0.42767632
  0.3383727   0.6485365  -0.31963384 -0.91161525 -0.6085069   0.4520803
  0.22863248  0.50233173 -0.4850589  -0.73530424 -0.20084564  0.2421846
  0.18445009  0.56838685 -0.18230604  0.27157733 -0.81602454 -1.0407478
  0.07344764  0.04872707 -0.2871886   0.25288028  0.05714323 -0.566729
 -0.16924308 -0.5878411   0.54115903  0.2743494  -0.4508098   0.9525985
  0.08284912  0.11392804 -0.798441    0.14463887  0.25419617  0.450397
  0.3117349   0.8645946   0.7458285   0.02277634  0.18972449  0.15366188
 -0.6390862  -0.00122656 -0.62477213 -0.58279526  0.4187462  

両者ともに同じベクトルですね。

## 課題:
単語「猫」を単語ベクトルで表してみましょう。  
また、skip-gramも試して結果を比較してみましょう。